# Assignment 3 - SMAI - DoubleMNIST and PermutedMnist

In [1]:
import os
import sys

# enter the Foldername here:
FOLDERNAME = "/home/richard/play/IIITH/sem5/smai/assignments/assignment-3-fine-man"

if FOLDERNAME is None or not os.path.exists(FOLDERNAME):
    FOLDERNAME = os.getcwd()

PATHNAME = f"{FOLDERNAME}"
sys.path.append(f"{FOLDERNAME}")

# DATA_FOLDER = os.path.join(FOLDERNAME, "SMAI-Dataset-release/IIIT-CFW")
DATA_FOLDER = os.path.join(FOLDERNAME, "datasets")
YAML_FOLDER = os.path.join(FOLDERNAME, "yaml-files")
print(DATA_FOLDER)
print(YAML_FOLDER)

/home/vanshg/play/IIIITH/sem5/smai/assignments/assignment-3-fine-man/datasets
/home/vanshg/play/IIIITH/sem5/smai/assignments/assignment-3-fine-man/yaml-files


In [2]:
# some magic so that the notebook will reload external python modules;
# see https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import cv2
import scipy
import scipy.io
import os
from random import randrange
from matplotlib import pyplot as plt
from PIL import Image
import wandb
import yaml

from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
from torch.utils.data import random_split

import torchvision
from torchvision import transforms

In [4]:
from src_torch import *
from src_torch.classifiers import *

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: vanshg. Use `wandb login --relogin` to force relogin


True

## (5.1) Multi-digit Recognition on Multi-MNIST Dataset

### (5.1.0) Data Loading, Preprocessing and Visualization

In [6]:
DOUBLE_MNIST_FOLDER = os.path.join(DATA_FOLDER, "DOUBLE_MNIST")

train_img_paths = []
val_img_paths = []
test_img_paths = []

# Saving path of all images in train set
for img_folder in glob.glob(os.path.join(DOUBLE_MNIST_FOLDER, "train/*")):
    label = int(os.path.basename(img_folder))
    for img_file in glob.glob(os.path.join(img_folder, "*")):
        train_img_paths.append(img_file)

# Saving path of all images in val set
for img_folder in glob.glob(os.path.join(DOUBLE_MNIST_FOLDER, "val/*")):
    label = int(os.path.basename(img_folder))
    for img_file in glob.glob(os.path.join(img_folder, "*")):
        val_img_paths.append(img_file)

# Saving path of all images in test set
for img_folder in glob.glob(os.path.join(DOUBLE_MNIST_FOLDER, "test/*")):
    label = int(os.path.basename(img_folder))
    for img_file in glob.glob(os.path.join(img_folder, "*")):
        test_img_paths.append(img_file)

print(f"Length of Training data: {len(train_img_paths)}")
print(f"Length of Validation data: {len(val_img_paths)}")
print(f"Length of Testing Data: {len(test_img_paths)}")

Length of Training data: 64000
Length of Validation data: 16000
Length of Testing Data: 20000


In [28]:
train_dataset = DoubleMNIST(train_img_paths)
val_dataset = DoubleMNIST(val_img_paths)
test_dataset = DoubleMNIST(test_img_paths)

## (5.2) - Permuted MNIST

### (5.2.0) Data Loading, PreProcessing and Visualizing

In [7]:
PERMUTED_MNIST_FILE = os.path.join(DATA_FOLDER, "permuted_mnist.npz")

data = np.load(PERMUTED_MNIST_FILE)

train_images = torch.tensor(data['train_images'])/255.0
train_images = train_images.unsqueeze(1)
train_labels = torch.tensor(data['train_labels'])
test_images = torch.tensor(data['test_images'])/255.0
test_images = test_images.unsqueeze(1)
test_labels = torch.tensor(data['test_labels'])

complete_train_dataset = TensorDataset(train_images, train_labels)
test_dataset = TensorDataset(test_images, test_labels)

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([60000])
torch.Size([10000, 1, 28, 28])
torch.Size([10000])


#### Splitting into Train/Val Data

In [8]:
val_ratio = 0.3
val_size = int(val_ratio * len(complete_train_dataset))
train_size = len(complete_train_dataset) - val_size

train_dataset, val_dataset = random_split(complete_train_dataset, [train_size, val_size])

print(f"Length of Training Data: {len(train_dataset)}")
print(f"Length of Validation Data: {len(val_dataset)}")
print(f"Length of Testing Data: {len(test_dataset)}")

Length of Training Data: 42000
Length of Validation Data: 18000
Length of Testing Data: 10000


#### (5.2.1) - MLP on Permuted MNIST

In [9]:
print(train_dataset[0][0].shape)

torch.Size([1, 28, 28])


In [45]:
class SimpleMLP(nn.Module):
    def __init__(
            self,
            input_dim,
            hidden_dims,
            num_classes,
            flatten_first=False, # Whether to flatten the input first
            last_activation=False, # activation to be used after last layer
    ):
        super(SimpleMLP, self).__init__()
        self.input_dim = input_dim
        self.num_layers = len(hidden_dims) + 1
        self.hidden_dims = hidden_dims
        self.flatten_first = flatten_first
        self.last_activation = last_activation 
        modules = []

        # first layer
        in_dim = input_dim
        if self.num_layers > 1:
            output_dim = hidden_dims[0]
            modules.append(nn.Linear(in_dim, output_dim))
            modules.append(nn.ReLU())
            in_dim = output_dim
        
        # Intermediate layers
        for i in range(1, self.num_layers - 1):
            out_dim = hidden_dims[i]
            modules.append(nn.Linear(in_dim, out_dim))
            modules.append(nn.ReLU())
            in_dim = out_dim
        
        # Last Layer
        out_dim = num_classes
        modules.append(nn.Linear(in_dim, out_dim))
        if self.last_activation:
            modules.append(nn.ReLU())

        self.linears = nn.ModuleList(modules)
        
    def forward(self, x):
        N = x.shape[0] # Number of samples
        if self.flatten_first:
            out = x.reshape(N, -1)
        else:
            out = x

        for module in self.linears:
            out = module(out)
        
        return out

In [46]:
model_config = {
    "input_dim": 28*28,
    "hidden_dims": [],
    "num_classes": 10,
    "flatten_first": True
}

mlp_model = SimpleMLP(**model_config)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.01)

print(mlp_model)

train_config = {
    "device": device,
    "print_every": 100
}

_ = train(mlp_model, criterion, optimizer, train_dataset, val_dataset, **train_config)

SimpleMLP(
  (linears): ModuleList(
    (0): Linear(in_features=784, out_features=10, bias=True)
  )
)

Model is on device: cuda

Number of Iterations Per Epoch: 420


Iteration: 100/4200 | loss = 0.4231
Iteration: 200/4200 | loss = 0.3212
Iteration: 300/4200 | loss = 0.2445
Iteration: 400/4200 | loss = 0.4077
Epoch: 1 | Train Accuracy: 92.005 | Val Accuracy: 91.333|  Train loss: 0.2873 | Val loss: 0.3076

Iteration: 500/4200 | loss = 0.4777
Iteration: 600/4200 | loss = 0.3131
Iteration: 700/4200 | loss = 0.2633
Iteration: 800/4200 | loss = 0.2678
Epoch: 2 | Train Accuracy: 92.074 | Val Accuracy: 90.894|  Train loss: 0.2826 | Val loss: 0.3192

Iteration: 900/4200 | loss = 0.3209
Iteration: 1000/4200 | loss = 0.3706
Iteration: 1100/4200 | loss = 0.3235
Iteration: 1200/4200 | loss = 0.3436
Epoch: 3 | Train Accuracy: 92.748 | Val Accuracy: 91.667|  Train loss: 0.2623 | Val loss: 0.3046

Iteration: 1300/4200 | loss = 0.2735
Iteration: 1400/4200 | loss = 0.1594
Iteration: 1500/4200 | loss = 0.2525
Iteration: 1600/4200 | loss = 0.2847
Epoch: 4 | Train Accuracy: 92.440 | Val Accuracy: 90.972|  Train loss: 0.2598 | Val loss: 0.3190

Iteration: 1700/4200 | lo

In [49]:
# Testing performance on Test Dataset
test_acc, test_loss = evaluate(mlp_model, test_dataset, device=device)
print(f"Test Accuracy: {test_acc*100:.4f} | Test Loss: {test_loss:.4f}")

Test Accuracy: 91.3100 | Test Loss: 0.3261


##### (5.2.1.2) Hyperparameter Tuning with Wandb

#### (5.2.2) CNN on Permuted MNIST

In [51]:
cnn_model = SimpleCNN(kernel_size=5, num_channels=128, stride=1, dropout=0.1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.01)

train_config = {
    "device": device,
    "print_every": 100
}

_ = train(cnn_model, criterion, optimizer, train_dataset, val_dataset, **train_config)


Model is on device: cuda

Number of Iterations Per Epoch: 420


Iteration: 100/4200 | loss = 0.3293
Iteration: 200/4200 | loss = 0.3878
Iteration: 300/4200 | loss = 0.3389
Iteration: 400/4200 | loss = 0.3546
Epoch: 1 | Train Accuracy: 95.105 | Val Accuracy: 93.767|  Train loss: 0.1590 | Val loss: 0.2003

Iteration: 500/4200 | loss = 0.3239
Iteration: 600/4200 | loss = 0.2299
Iteration: 700/4200 | loss = 0.1933
Iteration: 800/4200 | loss = 0.3872
Epoch: 2 | Train Accuracy: 95.840 | Val Accuracy: 94.228|  Train loss: 0.1342 | Val loss: 0.1933

Iteration: 900/4200 | loss = 0.2999
Iteration: 1000/4200 | loss = 0.1070
Iteration: 1100/4200 | loss = 0.2478
Iteration: 1200/4200 | loss = 0.1778
Epoch: 3 | Train Accuracy: 95.967 | Val Accuracy: 94.250|  Train loss: 0.1280 | Val loss: 0.1990

Iteration: 1300/4200 | loss = 0.1250
Iteration: 1400/4200 | loss = 0.0926
Iteration: 1500/4200 | loss = 0.2651
Iteration: 1600/4200 | loss = 0.3944
Epoch: 4 | Train Accuracy: 96.245 | Val Accuracy: 94.061|  Train loss: 0.1167 | Val loss: 0.2157

Iteration: 1700/4200 | lo

In [52]:
test_acc, test_loss = evaluate(cnn_model, test_dataset, device=device)
print(f"Test Accuracy: {test_acc*100:.4f} | Test Loss: {test_loss:.4f}")

Test Accuracy: 94.0900 | Test Loss: 0.2333


## (5.3) Analysis